In [ ]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.

In [ ]:
import pandas as pd
import numpy as np
# python -m pip install yahoo-finance; pip install yahoo-finance; pip install yfinance --upgrade --no-cache-dir
# import yfinance as yf
# pip install --upgrade ta; pip install ta
# import ta as ta
import sklearn as sk
from sklearn import preprocessing
from scipy.stats import t
import tensorflow as tf
from datetime import date, datetime, timedelta
# from arch import arch_model
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE
from neuralforecast.tsdataset import TimeSeriesDataset, TimeSeriesLoader
from numpy.random import seed
from random import randrange
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic

In [ ]:
from neuralforecast.models import NBEATS

In [ ]:
!pip install yahoo-finance

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.4 MB/s eta 0:00:00
  Created wheel for yahoo-finance: filename=yahoo_finance-1.4.0-py3-none-any.whl size=7217 sha256=b7645ca417a8a5d60fdba312ee4350e32728c3faf3b6b8d7deae196056cdb98c
  Stored in directory: /root/.cache/pip/wheels/f6/a9/34/f1aaa343d0861148f79a9df08f380e4dbbdbe27b7ba1e0e84c
Successfully built yahoo-finance


In [ ]:
import yfinance as yf

In [ ]:
snp = yf.download("^GSPC",start='2008-01-01', end='2024-01-25', progress=False)
snp

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,1467.969971,1471.770020,1442.069946,1447.160034,1447.160034,3452650000
2008-01-03,1447.550049,1456.800049,1443.729980,1447.160034,1447.160034,3429500000
2008-01-04,1444.010010,1444.010010,1411.189941,1411.630005,1411.630005,4166000000
2008-01-07,1414.069946,1423.869995,1403.449951,1416.180054,1416.180054,4221260000
2008-01-08,1415.709961,1430.280029,1388.300049,1390.189941,1390.189941,4705390000
...,...,...,...,...,...,...
2024-01-18,4760.100098,4785.790039,4740.569824,4780.939941,4780.939941,4019000000
2024-01-19,4796.279785,4842.069824,4785.870117,4839.810059,4839.810059,4287200000
2024-01-22,4853.419922,4868.410156,4844.049805,4850.430176,4850.430176,4297610000


In [ ]:
snp.index = pd.to_datetime(snp.index)

In [ ]:
snp = snp.resample('D').fillna(method='ffill')

In [ ]:
snp

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,1467.969971,1471.770020,1442.069946,1447.160034,1447.160034,3452650000
2008-01-03,1447.550049,1456.800049,1443.729980,1447.160034,1447.160034,3429500000
2008-01-04,1444.010010,1444.010010,1411.189941,1411.630005,1411.630005,4166000000
2008-01-05,1444.010010,1444.010010,1411.189941,1411.630005,1411.630005,4166000000
2008-01-06,1444.010010,1444.010010,1411.189941,1411.630005,1411.630005,4166000000
...,...,...,...,...,...,...
2024-01-20,4796.279785,4842.069824,4785.870117,4839.810059,4839.810059,4287200000
2024-01-21,4796.279785,4842.069824,4785.870117,4839.810059,4839.810059,4287200000
2024-01-22,4853.419922,4868.410156,4844.049805,4850.430176,4850.430176,4297610000


In [ ]:
# Ensure the 'Price' column is numeric
# test_df['Price'] = pd.to_numeric(test_df['Price'], errors='coerce')

# If there are any None values after conversion, this will remove those rows
# df.dropna(subset=['Price'], inplace=True)

window = 21
trd_days = 252
ann_factor = window / trd_days

# Calculate log returns
snp['log_return'] = np.log(snp['Close'] / snp['Close'].shift(1))

# Calculate realized variance
rlz_var = snp['log_return'].rolling(window=window).var() * ann_factor

# Calculate realized volatility
rlz_vol = np.sqrt(rlz_var)

# You might want to store the realized volatility in the DataFrame
snp['realized_volatility'] = rlz_vol

In [ ]:
snp = snp.reset_index(drop=False)
snp

,Date,Open,High,Low,Close,Adj Close,Volume,log_return,realized_volatility
0,2008-01-02,1467.969971,1471.770020,1442.069946,1447.160034,1447.160034,3452650000,NaN,NaN
1,2008-01-03,1447.550049,1456.800049,1443.729980,1447.160034,1447.160034,3429500000,0.000000,NaN
2,2008-01-04,1444.010010,1444.010010,1411.189941,1411.630005,1411.630005,4166000000,-0.024858,NaN
3,2008-01-05,1444.010010,1444.010010,1411.189941,1411.630005,1411.630005,4166000000,0.000000,NaN
4,2008-01-06,1444.010010,1444.010010,1411.189941,1411.630005,1411.630005,4166000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
5862,2024-01-20,4796.279785,4842.069824,4785.870117,4839.810059,4839.810059,4287200000,0.000000,0.001589
5863,2024-01-21,4796.279785,4842.069824,4785.870117,4839.810059,4839.810059,4287200000,0.000000,0.001589
5864,2024-01-22,4853.419922,4868.410156,4844.049805,4850.430176,4850.430176,4297610000,0.002192,0.001591
5865,2024-01-23,4856.799805,4866.479980,4844.370117,4864.600098,4864.600098,3912800000,0.002917,0.001536


In [ ]:
snp = snp.dropna()
snp

,Date,Open,High,Low,Close,Adj Close,Volume,log_return,realized_volatility
21,2008-01-23,1310.410034,1339.089966,1270.050049,1338.599976,1338.599976,3241680000,0.021216,0.003755
22,2008-01-24,1340.130005,1355.150024,1334.310059,1352.069946,1352.069946,5735300000,0.010012,0.003848
23,2008-01-25,1357.319946,1368.560059,1327.500000,1330.609985,1330.609985,4882250000,-0.015999,0.003677
24,2008-01-26,1357.319946,1368.560059,1327.500000,1330.609985,1330.609985,4882250000,0.000000,0.003677
25,2008-01-27,1357.319946,1368.560059,1327.500000,1330.609985,1330.609985,4882250000,0.000000,0.003677
...,...,...,...,...,...,...,...,...,...
5862,2024-01-20,4796.279785,4842.069824,4785.870117,4839.810059,4839.810059,4287200000,0.000000,0.001589
5863,2024-01-21,4796.279785,4842.069824,4785.870117,4839.810059,4839.810059,4287200000,0.000000,0.001589
5864,2024-01-22,4853.419922,4868.410156,4844.049805,4850.430176,4850.430176,4297610000,0.002192,0.001591
5865,2024-01-23,4856.799805,4866.479980,4844.370117,4864.600098,4864.600098,3912800000,0.002917,0.001536


In [ ]:
snp['unique_id'] = '1'
snp.rename(columns={'Date':'ds'}, inplace=True)
snp.rename(columns={'realized_volatility':'y'}, inplace=True)

<ipython-input-14-e26a8bc893c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp['unique_id'] = '1'
<ipython-input-14-e26a8bc893c6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp.rename(columns={'Date':'ds'}, inplace=True)
<ipython-input-14-e26a8bc893c6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp.rename(columns={'realized_volatility':'y'}, inplace=True)


In [ ]:
train = snp.loc[:'2016-01-01'].reset_index(drop=False)
# val = df.loc['2022-11-30':'2022-12-30'].reset_index(drop=False)
test = snp.loc['2016-01-01':].reset_index(drop=False)

In [ ]:
model = NBEATS(h=len(test), input_size=1,
                loss=MQLoss(level=[90]),
                #  loss = DistributionLoss(distribution='Poisson', level=[90])
                # scaler_type='minmax',
                #  scaler_type='standard',
                learning_rate= 0.001,
                stack_types=['identity', 'identity'],#,'trend', 'seasonality'
                n_blocks=[1,1],
                mlp_units= [[512, 512], [512, 512]],
                # windows_batch_size=60,
                num_lr_decays=5,
                # val_check_steps=20,
                n_harmonics=0, n_polynomials=0,

                max_steps=200,
                # early_stop_patience_steps=2,

                )

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
fcst = NeuralForecast(
    models=[model],
    freq='D'
  )

In [ ]:
fcst.fit(df=train)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MQLoss        | 3     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 12.9 M
-----------------------------------------------
12.9 M    Trainable params
3         Non-trainable params
12.9 M    Total params
51.616    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=200` reached.


In [ ]:
forecasts = fcst.predict()

/usr/local/lib/python3.10/dist-packages/neuralforecast/tsdataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.temporal = torch.tensor(temporal, dtype=torch.float)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:184: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import make_scorer

#metric used to measure the model (Root mean squared error)
def rmse1(y_actual, y_pred):
    rmse = np.sqrt(mean_squared_error(y_actual,y_pred))
    return rmse
rmse_cv = make_scorer(rmse1, greater_is_better=False)

def mse1(y_actual, y_pred):
    mse = mean_squared_error(y_actual,y_pred)
    return mse
mse_cv = make_scorer(mse1, greater_is_better=False)

def mae1(y_actual, y_pred):
    mae = mean_absolute_error(y_actual,y_pred)
    return mae
mae_cv = make_scorer(mae1, greater_is_better=False)

def smape1(a, f):
    return 1/len(a) * np.sum(2*np.abs(f-a)/(np.abs(a)+np.abs(f))*100)


In [ ]:
forecast_rmse = rmse1(test['y'], forecasts['NBEATS-median'])
print(forecast_rmse)
forecast_mse = mse1(test['y'], forecasts['NBEATS-median'])
print(forecast_mse)
forecast_mae = mae1(test['y'], forecasts['NBEATS-median'])
print(forecast_mae)

0.024991746891106746
0.0006245874126691437
0.014568029194018488
